In [ ]:
from autumn.projects.sm_covid2.common_school.runner_tools import INCLUDED_COUNTRIES
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt 
from autumn.projects.sm_covid2.common_school.output_plots.multicountry import plot_ll_comparison
import arviz as az
full_iso3_list = list(INCLUDED_COUNTRIES['all'].keys())


In [ ]:
analysis_folders = {
    "main": Path.cwd() / "31747883_full_analysis_26Sep2023_main",
    "increased_hh_contacts": Path.cwd() / "31902886_full_analysis_05Oct2023_increased_hh_contacts",
    "no_google_mobility": Path.cwd() / "31915437_full_analysis_05Oct2023_no_google_mobility"
}

### Collect idata x-arrays for all analyses

In [ ]:
idatas = {}
for iso3 in ["BLR"]: # full_iso3_list:
    idatas[iso3] = {}
    for analysis, folder in analysis_folders.items():
        idata_path = folder / iso3 / "idata.nc"
        if Path.exists(idata_path):
            idatas[iso3][analysis] = az.from_netcdf(idata_path)

### Burn unused parameter sets and store smaller x-arrays

In [ ]:
outpath = Path.cwd() / "burnt_idatas"

for iso3 in ["BLR"]: # full_iso3_list:
    for analysis in analysis_folders:
        idata = idatas[iso3][analysis]
        chain_length = idata.sample_stats.sizes['draw']
        burnt_idata = idata.sel(draw=range(20000, chain_length))  # Discard burn-in

        az.to_netcdf(burnt_idata, outpath / f"b_idata_{iso3}_{analysis}.nc")

    

# Computation of likelihood extras now done on MASSIVE

### Load results of likelihood extra computations   

In [ ]:
lle_data_dir = Path.cwd() / "lle_csvs"
lle_dfs = {}
for iso3 in full_iso3_list:
    lle_dfs[iso3] = {}
    for analysis in analysis_folders:
        df = pd.read_csv(lle_data_dir / f"lle_{iso3}_{analysis}.csv")
        df["iso3"] = [iso3] * len(df)
        df["analysis"] = [analysis] * len(df)
        lle_dfs[iso3][analysis] = df

lle_dfs_list = [lle_dfs[iso3][analysis] for iso3 in full_iso3_list for analysis in analysis_folders]
df = pd.concat(lle_dfs_list)

In [ ]:
FILE_FORMATS = ["png", "pdf"]

In [ ]:
for output in ["logposterior", "ll_extra_ll"]:
    fig = plot_ll_comparison(df, output)
    plt.close()
    for file_format in FILE_FORMATS:
        fig.savefig(Path.cwd() / "analyses_compare_plots" / f"ll_compare_{output}.{file_format}")
